# Contents


- [Introduction](#the_destination)
  * [Why deep learning and pdes](#sub-heading)
  * [Applications](#sub-heading)
  * [Solving PDEs with DL, overview](#sub-sub-heading)
- [DL, a modern recipe](#heading-1)
  * [DL basics](#sub-heading-1)
  * [Backpropagation (it's just good'ol adjoint method)](#sub-heading-1)
  * [Going deep, main problems, main solutions](#sub-heading-1)
  * [Practicioner recipe](#sub-heading-1)
- [Learning from data](#heading-2)
  * [Problem Statement](#sub-heading-2)
  * [Kernel Learning](#sub-sub-heading-2)
  * [Case study Allen Cahn](#sub-sub-heading-2)
  * [Multiple scales](#whatever)
  * [Results](#hehe)
      + [Validation dataset time predictions](#hhoo)
      + [Phase plots](#hehe)
      + [Time errors](#hoho)
      + [Resolution dependency](#hoho)
      + [Multiiple scales](#hehe)
  * [Code disection](#huhu)
- [Unsupervised Learning of the PDE, PINNS](#huhu)

# Introduction
## Why Deep Learning and PDEs


Deep learning fenomenal tool to learn arbitrarly complex functions byembedding data intro a continuous manifold that encode in much  lower dimensional space the main directions of variation of the data diistribution
<img width="600" height="600" style = "float : right" src="media/Intro_manifold.png">

If this works with data with a much tangible stucture as music and images, it is expected to perform even better with mathematical objects that are much more likely to live in a continuous smooth manifold, with principal modes of evolution, as it is the case with the solution of PDEs equations 
<img width="600" height="600" style = "float : left" src="media/intro_modes.gif">


# Introduction
## Applications

The biggest contributions of DL to the solution is the potential reduction in orders of magnitude the time complexity of finding a solution.This entails inmediate applications:
   * Inverse problems
   * Control processes
   * Computer graphics
   * Fast prototyping, assisted engineering.

Other contributions beside this should be mentioned
    - Explainability green ops, coherent structures etc
    - Data assimilation, data library, bessel etc

# Introduction
## How do we solve PDEs with deep learning
### Two general approaches

   - Supervised learning approach: Sample data from the population of solutions, and make the nerual network learn the mapping $NN: parameter \rightarrow solution$
   - Weighted residuals approach: Reparametrice $ N_{p}(y,\frac{\partial y}{\partial x},...) = 0 $ with a neural network $\hat{y}(x,\theta)$ and minimize the residues of the associated functional along with the BCs.

# Deep Learning, a modern recipe
## Neural network basics
### Dont fix the basis, learn it

Example, least squares regression problem $ \underset{w}{argmin}L(w) = \underset{w}{argmin}\sum_{Data}|y-\hat{y}(x,w)|^{2}$
 * Feature engenering, "shallow learning": $\hat{y} = \sum_{i}w_{i}\phi_{i}(x) $
 * Neural networks : $\hat{y} = \sum_{i}w_{i}\phi_{i}(x, w) $
 
Instead of just addition as a building mechanism, NN use composition of blocks of non linear functions and linear applications called layers.
\begin{equation}
\phi(w,x) = \sigma(W_{L} \sigma(W_{L-1}....\sigma(W_{1}X) )
\end{equation}
 
To optimize we need to calculate the gradients $\partial_{w}L$ respect to the weights of every layer. The algorithm that does it is a special implementation  of the adjoint method, called backpropagation.



# Deep Learning, a modern recipe
## Neural network basics
### Calculating gradients
#### Reformulation as a constrained optimization problem

\begin{array}{lc}
\mbox{J}: \underset{p}{\mbox{minimize}} & \sum(x_{Data}-\hat{x}(x_{input},p))^{2}  \\
\end{array}
\begin{array}{lm}
\mbox{G}: &   \begin{cases} 
      \hat{x} = \sigma(Wx_{k})  \\
      x_{k} = \sigma(W_{k}x_{k-1}) \\
      \vdots \\
      x_{2} = \sigma(W_{2}x_{1})\\
      x_{1} = \sigma(W_{1}x_{input}) \\
   \end{cases} \\
\end{array}

# Deep Learning, a modern recipe
## Neural network basics
### Calculating gradients
#### Adjoint method
\begin{array}{lc}
\underset{p}{\mbox{minimize}} & J(x_{T},p)  \\
\end{array}

\begin{array}{lcm}
\mbox{forward system} & g(x,p) = 0 = &   \begin{cases} 
      g(x_{T},x_{T-1},p) = 0  \\
      g(x_{T-1},x_{T-2},p) = 0 \\
      \vdots \\
      g(x_{2},x_{1},p) = 0\\
      g(x_{1},x_{0},p) = 0 \\
      g(x_{0},p) = 0
   \end{cases} \\
\end{array}


\begin{equation}
J_{aug}(x_{T},p) = J(x_{T},p) + \lambda_{T}g(x_{T},x_{T-1},p) + \lambda_{T-1}g(x_{T-1},x_{T-2},p) \ldots \lambda_{1}g(x_{1},x_{0},p)+ \lambda_{0}g(x_{0},p)
\end{equation}


\begin{array}{lm}
\mbox{backward/adjoint system} & \begin{cases}
        \partial_{x}J+\lambda_{T}\partial_{x_{T}}g_{T} = 0 \\
        \lambda_{T}\partial_{x_{T-1}}g_{T}+\lambda_{T-1}\partial_{x_{T-1}}g_{T-1} =0 \\
        \vdots \\
        \lambda_{1}\partial_{x_{0}}g_{1}+\lambda_{0}\partial_{x_{0}}g_{0} = 0 \\
        \end{cases} \\
\end{array}

\begin{array}{lm}
\mbox{Gradient} & \begin{equation}
        d_{p} J_{aug} = \partial_{p} J +
        \lambda_{T}\partial_{p}g_{T}+\lambda_{T-1}\partial_{p}g_{T-1}+\ldots +
        \lambda_{1}\partial_{p}g_{1}+
        \lambda_{0}\partial_{p}g_{0}
        \end{equation}
\end{array}

if we use $g_{L} =x_{L}-\sigma(W_{L}x_{L-1}) = 0$ and clear $\lambda$ we get the $Backpropagation$ algorithm

# Deep Learning, a modern recipe
## Neural network basics
### Calculating gradients
#### Adjoint method in time dependent problems, just for clarity


\begin{array}{lc}
\underset{p}{\mbox{minimize}} & J(x(t),p) = \int_{0}^{T} f(x(t),p) \,dt  \\
\end{array}

\begin{array}{lc}
\mbox{forward} & \begin{cases} 
      g(\dot{x},x,p) = 0  \\
      g_{0}(x_{0},p) = 0
   \end{cases} \\
\end{array}

\begin{array}{lc}
\mbox{backward/adjoint system} & \begin{cases} 
       \dot{\lambda(\tau)}\partial_{\dot{x}}g - \lambda(\tau)\dot{\partial_{\dot{x}}}g + \lambda(\tau)\partial_{x}g = -\partial_{x}f  \\
     \lambda(\tau = 0) = 0 \\
      \mu=\lambda\partial_{\dot{x}}g\partial_{x}g_{0}^{-1}
   \end{cases} \\
\end{array}

\begin{array}{lc}
\mbox{Gradient} & \begin{equation}
 d_{p}J_{aug} = \int_{0}^{T}  \partial_{p}f+\lambda\partial_{p}gdt+\mu\partial_{p}g_{0}\Bigr|_{t = 0}
 \end{equation} \\
\end{array}

# Deep Learning, a modern recipe
## Main problems, main solutions
The main problem we encounter as we try to fit networks with increasing numbers of layers, is the explosion/vanishing variance of the activations and gradients of different layers.
 * PROBLEM: Decaying and exploding variance. Main techniques to stabilize the network and control variance:
  * Proper initialization of weights.
  * Normalization ( mostly batchnorm).
  
  
 * PROBLEM: Saturation.
  * Residual connections
  
  
 * PROBLEM: Optimization in a bumpy loss landscape.
  * Stochastic gradient descent with momentum , adaptative and annealed learning rate.
  
  
 * PROBLEM: Overfitting.
  * Depending on the application it is not that much of a problem.
  * Regularization, L2, dropout...

# Deep Learning, a modern recipe
## Recipe for building block/layer  and optimizer

Use resnet (with different lengths) as a template and use as much capacity as you can afford.

Main building block consist in layers with residual skip connections and batch normalization. If using Relu weights must be initializated with the proper scale ( He initialization). Scale input and labels and use Adam optimization with default parameters (pytorch), and small (1e-5) weight decay and reduce on plateau scheduler.

<img width="300" height="300" style = "float : left" src="media/basic_block.jpg">


# Learning from data
## Problem statement

We consider the PDE $P$ with BC as a mapping $\psi$ between function spaces where $X$ is the parameter space and $Y$ the solution space.
$$P_{X}(y) = 0$$

$F$ and $G$ are the operators that project the data to a discrete space. The symbol $\varphi$ represent the mapping in the discrete space.


![image.png](media/scheme_operator.jpg)

If we work directly in the discretized space, we'll model the mapping with a convolutional neural network by minimizing:
$$ \underset{\theta}{argmin} \underset{x \sim \mu}{E}(Cost(\varphi_{solver}(x)-\hat{\varphi}(x,\theta))$$

If we work in a function space we'll minimize:
$$ \underset{\theta}{argmin} \underset{x \sim \mu}{E}(Cost(\psi_{solver}(x)-\hat{\psi}(x,\theta))$$

Both methods work with discrete data , but in the first case , we are learning directly a mapping  in $R^{N_{grid}}$ while in the second case we first project to a function space (fourier transform), we learn the mapping there and transform back to the discretized space.


# Learning from data
## Case study : Evolutive system, Allen Cahn, spinodal decomposition

$$
\begin{array}{l}
    \partial_{t}u-M(\Delta u -\frac{1}{\epsilon^{2}}(u^{2}-1)u) = 0 \\
     u,\nabla u |_{\partial \Omega} \quad periodic \\
     u(0,x,y) = u_{0}(x,y)\\
     x,y\in[0,1]
\end{array}
$$


  Gibs free energy vs phase            |  Initial condition, small fluctuations that trigger the decomposition
:-------------------------:|:-------------------------:
<img src="media/gibbs_potential.jpg" alt="drawing" width="400" height="400"  />  |  <img src="media/noise.png" alt="drawing" width="400" height="400"  />


# Learning from data
## Case study : Evolutive system, Allen Cahn, spinodal decomposition

  Simulations samples      |  $ M = 1,\epsilon = 0.01, T = 200 dt$
:-------------------------:|:-------------------------:
<img src="media/sample1.gif"   />  |  <img src="media/sample2.gif"   />
<img src="media/sample3.gif"   />  |  <img src="media/sample4.gif"   />



# Learning from data
## Case study : Evolutive system, Allen Cahn, spinodal decomposition

This is an interesting problem to learn beacuse without being chaotic , it exhibits multiple spatial and temporal timescales that must be solved simultaneously to accurattely predict long term behaviour.

We have a extremely fast destabilization at the beggining that is followed by a slow evolution guided by the interface advances. Even if the coalescence stage is generally slow, when two drops are close to each other, the blending is fast, so it must be captured with a sufficiently small time step


   time evolution   |  $E(abs(phase))\quad vs \quad time$
:-------------------------:|:-------------------------:
<img src="media/sample_decomp.gif" width = "400" height = "400"  />  |  <img src="media/sample_decomp_phase.png"  width = "400"  height = "400" />

# Learning from data
## Models architecture

   Image-Image CNN   |  Fourier Neural Operator
:-------------------------:|:-------------------------:
<img src="media/unetlike.png" width = "800" height = "300"  />  |  <img src="media/fourier_operator.jpg"  width = "800"  height = "500" />



# Learning from data
## Training


The mapping we'll try to learn is $\Psi: u_{T-\Delta t}\rightarrow u_{T} $ with the goal of applying it recurrently to predict evolution times much longer than $\Delta$. As NN are not constrained by the time integration errors of traditional schemes, REFERENCE MULTISCALE, large $\Delta t, 6*dt \quad solver$.
The objetive is to minize:
$$\underset{\theta}{argmin}\underset{u_{0},T}{E}(|u_{T+\Delta t}-\hat{u}_{T+\Delta t}(\theta, u_{T})|^{2})$$

In both models, after the validation error crosed a treshold, more time steps ahead are predicted recurrently to increase stability, up to 4 timesteps.



# Learning from data
## Code dissection


